# <font color=orange>**Evrisimsel Sinir Aglari ile Nesne Tespiti**</font> <font color=gray>**(CNN)**</font>

## <font color=orange>**Maksimum Olmayan Bastirma**</font> <font color=gray>**-**</font>

* Daha kucuk ortusen sinirlayici kutulari yok sayar ve yalnizca daha buyuk olanlari dondurur.

* Kutular arasinda **Birlik Uzerinden Kesisme <font color=red>(Intersection over Union)</font> degeri hesaplanir.

* Belirli bir esik degerinin altinda kalan kutular elenir.

In [1]:
import cv2
import numpy as np

In [3]:
def non_maxi_supression(boxes, probs= None, overlapThresh=(0.3)):
    if len(boxes) == 0:
        return[]
    
    if boxes.dtype.kind == 'i':
        boxes = boxes.asstype('float')
        
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    
    # Alani bulalim:
    area = (x2 - x1 + 1)*(y2 - y1 + 1)
    
    idxs = y2
    # Olasilik degerleri:
    if probs is not None:
        idxs = probs
        
    idxs = np.argsort(idxs)
    pick = [] # Secilen kutular
    
    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
        
        # En buyuk ve en kucuk x ve y:
        xx1 = np. maximum(x1[i], x1[idxs[:last]])
        yy1 = np. maximum(y1[i], y1[idxs[:last]])
        xx2 = np. minimum(x2[i], x2[idxs[:last]])
        yy2 = np. minimum(y2[i], y2[idxs[:last]])
        
        # w, h bulma:
        w = np.maximum(0,xx2 - xx1 + 1)
        h = np.maximum(0,yy2 - yy1 + 1)
        
        # overlap 
        overlap = (w*h)/area[idxs[:last]]
        #threshold un altinda kalan indexleri silme:
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh))))
    
    return boxes[pick].astype('int')